# SharedPreferences - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu **SharedPreferences** w celu zapisania niewielkich ilości danych w pliku.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHZ0Mmt6cjJwYjVrYzE0NWdpbHNqYXM5ZjVvNzc1bmhqZm1zY2F5NSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/kd4nprniMKNt9fBZIv/giphy.gif" width="200" />

`SharedPreferences` to mechanizm używany do przechowywania danych w postaci *klucz-wartość*. Służy do zachowywania małych ilości danych, takich jak ustawienia aplikacji, preferencje użytkownika czy stan sesji. Jest często używany do przechowywania informacji, które nie wymagają zbyt skomplikowanego modelu danych ani relacyjnej bazy danych. Przechowuje dane w plikach `XML`, które są przechowywane wewnątrz katalogu aplikacji.
`SharedPreferences` a `DataStore`:
- Typ danych i bezpieczeństwo:
    - `SharedPreferences` przechowuje dane w plikach `XML` i obsługuje tylko typy prostych danych, takie jak liczby całkowite, ciągi znaków, wartości boolean itp. Jednak nie oferuje wbudowanej obsługi bardziej złożonych struktur danych.
    - `DataStore` obsługuje niestandardowe typy danych i zapewnia automatyczną obsługę konwersji do i z formatu `protobuf`.
- Wsparcie dla asynchroniczności:
    - `SharedPreferences` oferuje operacje synchroniczne, co może wpływać na wydajność aplikacji, szczególnie gdy operacje odczytu/zapisu danych są wykonywane w wątku głównym.
    - `DataStore` został zaprojektowany z myślą o asynchroniczności i wspiera Coroutines, dzięki czemu operacje odczytu/zapisu danych mogą być wykonywane asynchronicznie, co poprawia wydajność i responsywność aplikacji.
- Bezpieczeństwo wątkowe:
    - `SharedPreferences` nie jest wątkowo bezpieczne, co oznacza, że operacje odczytu/zapisu mogą powodować błędy synchronizacji, jeśli są wykonywane równocześnie przez wiele wątków.
    - `DataStore` posiada wbudowane mechanizmy bezpieczeństwa ze względu na wielowątkowość. Można go bezpiecznie używać w aplikacjach wielowątkowych bez konieczności dodatkowej synchronizacji.
- Obsługa zmian danych:
    - `SharedPreferences` nie oferuje wbudowanej obsługi reagowania na zmiany danych. Możemy jedynie odczytać aktualny stan.
    - `DataStore` umożliwia korzystanie z obiektu `Flow` lub `LiveData`, co pozwala na automatyczną obsługę zmian danych. Możemy zarejestrować obserwatora, który będzie otrzymywał powiadomienia o zmianach danych, co ułatwia reagowanie na aktualizacje danych w czasie rzeczywistym.
    
Aplikacja będzie kopią aplikacji z poprzedniego przykładu, dodajmy odpowiednie zależności
do bloku `dependencies`
```kotlin
dependencies {
    implementation (libs.androidx.fragment.ktx)
    implementation (libs.androidx.lifecycle.viewmodel.ktx)
    ...
}
```

do pliku `libs.versions.toml`

```kotlin
[versions]
fragmentKtx = "1.8.2"
lifecycleViewmodelKtx = "2.8.4"
...

[libraries]
androidx-fragment-ktx = { module = "androidx.fragment:fragment-ktx", version.ref = "fragmentKtx" }
androidx-lifecycle-viewmodel-ktx = { module = "androidx.lifecycle:lifecycle-viewmodel-ktx", version.ref = "lifecycleViewmodelKtx" }
...
```

Ponownie wykorzystamy *dummydata* do wygenerowania nazw użytkowników.

In [ ]:
object DataProvider {

    private val usernames = listOf(
        "CoolDude123",
        "SuperStar",
        "GamerGirl99",
        "TechMaster",
        "MusicLover",
        "FitnessFreak",
        "Traveler_21",
        "FoodieQueen",
        "NatureLover",
        "Fashionista",
        "Bookworm42",
        "MovieBuff",
        "AdventureSeeker",
        "PetLover_87",
        "SportsFanatic",
        "ArtisticSoul",
        "StarGazer",
        "YogaEnthusiast",
        "PhotographyPro",
        "DreamChaser",
        "BeachBum123",
        "CoffeeAddict",
        "GameChanger",
        "LaughOutLoud",
        "MindfulSoul",
        "HappyGoLucky",
        "TechGeek1",
        "FoodExplorer",
        "FitnessJunkie"
    )

    val username: String
        get() = usernames.random()
}

Następnie dodajmy repozytorium, podobnie jak w poprzednich przypadkach, konieczne jest wykorzystanie kontekstu.

In [ ]:
class UserRepository(application: Application) {

    private val sharedPref = application.getSharedPreferences("fileName", MODE_PRIVATE)

    private var _username: String = sharedPref.getString("username", "") ?: ""
    val username: String
        get() = _username

    fun add(newUsername: String) {
        val edit = sharedPref.edit()
        edit.putString("username", newUsername).apply()
        _username = newUsername
    }

    fun clear() {
        val edit = sharedPref.edit()
        edit.putString("username", "").apply()
        _username = ""
    }
}

- `private val sharedPref = application.getSharedPreferences("fileName", MODE_PRIVATE)` - Wykorzystuje `getSharedPreferences` na obiekcie `Application`, aby uzyskać dostęp do `SharedPreferences` o nazwie `fileName`. Argument `MODE_PRIVATE` wskazuje, że `SharedPreferences` są prywatne dla tej aplikacji i nie są dostępne dla innych aplikacji. Inne tryby:
    - `MODE_APPEND` - pozwala dopisywać kolejne elementy bez nadpisywania
    - `MODE_PRIVATE` - najczęściej wykorzystywany, dostęp do pliku tylko z poziomu aplikacji
    - `MODE_WORLD_READABLE` - zezwala innym aplikacjom na odczyt
    - `MODE_WORLD_WRITABVLE` - zezwala innym aplikacjom na zapis
- `private var _username: String = sharedPref.getString("username", "") ?: ""` - Jeśli wartość `username` odczytana jest jako `null`, używamy operatora elvis `?: ""`, aby ustawić domyślną wartość na pusty ciąg znaków.
- `val edit = sharedPref.edit()` - Tworzy nowy obiekt `SharedPreferences.Editor`, który jest używany do edycji `SharedPreferences`. Dzięki temu edytorowi możemy dokonać zmian, takich jak dodawanie, usuwanie lub aktualizowanie danych.
- `edit.putString("username", newUsername).apply()` 0 W tej linii używamy metody `putString()` na obiekcie edytora, aby dodać nową wartość do `SharedPreferences`. Metoda `putString()` przyjmuje dwa argumenty: klucz (w tym przypadku "username") i wartość (nowa nazwa użytkownika). Wartość zostaje zapisana pod podanym kluczem. Następnie wywołujemy `apply()`, aby zatwierdzić te zmiany.

Pozostałe części aplikacji nie różnią się od poprzedniego przykładu

In [ ]:
class UserViewModel(application: Application) : AndroidViewModel(application) {
    private val repository: UserRepository = UserRepository(application)
    private val _username: MutableStateFlow<String> = MutableStateFlow(repository.username)

    val username: StateFlow<String>
        get() = _username

    fun addUsername(username: String) {
        repository.add(username)
        _username.value = username
    }

    fun clearUsername() {
        repository.clear()
        _username.value = ""
    }
}

In [ ]:
class UserFragment : Fragment() {

     private lateinit var binding: FragmentUserBinding

    private val viewModel: UserViewModel by viewModels()

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentUserBinding.inflate(inflater)

        viewLifecycleOwner.lifecycleScope.launch {
            viewModel.username.collectLatest { user ->
                binding.textView.text = "Current saved user: $user"
            }
        }

        binding.addButton.setOnClickListener { viewModel.addUsername(DataProvider.username) }
        binding.clearButton.setOnClickListener { viewModel.clearUsername() }

        return binding.root
    }
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHZ0Mmt6cjJwYjVrYzE0NWdpbHNqYXM5ZjVvNzc1bmhqZm1zY2F5NSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/kd4nprniMKNt9fBZIv/giphy.gif" width="200" />